In [32]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler


In [33]:
data=pd.read_csv('train-vals.csv')
#data=pd.read_csv('test-vals.csv')

In [34]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Data columns (total 40 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     59400 non-null  int64  
 1   amount_tsh             59400 non-null  float64
 2   date_recorded          59400 non-null  object 
 3   funder                 55765 non-null  object 
 4   gps_height             59400 non-null  int64  
 5   installer              55745 non-null  object 
 6   longitude              59400 non-null  float64
 7   latitude               59400 non-null  float64
 8   wpt_name               59400 non-null  object 
 9   num_private            59400 non-null  int64  
 10  basin                  59400 non-null  object 
 11  subvillage             59029 non-null  object 
 12  region                 59400 non-null  object 
 13  region_code            59400 non-null  int64  
 14  district_code          59400 non-null  int64  
 15  lg

In [35]:
int_cols = ['gps_height','population','amount_tsh','longitude','latitude','construction_year']
for j in int_cols:
  data[j].replace(0.0, np.nan, inplace=True)
data.isnull().sum()

id                           0
amount_tsh               41639
date_recorded                0
funder                    3635
gps_height               20438
installer                 3655
longitude                 1812
latitude                     0
wpt_name                     0
num_private                  0
basin                        0
subvillage                 371
region                       0
region_code                  0
district_code                0
lga                          0
ward                         0
population               21381
public_meeting            3334
recorded_by                  0
scheme_management         3877
scheme_name              28166
permit                    3056
construction_year        20709
extraction_type              0
extraction_type_group        0
extraction_type_class        0
management                   0
management_group             0
payment                      0
payment_type                 0
water_quality                0
quality_

In [36]:
district_code_group = data.groupby(['region', 'district_code'])
region_group = data.groupby(['region'])

null_cols_int = [ "amount_tsh", "gps_height", "population","latitude","longitude"]
for i in null_cols_int:
  data[i].fillna(district_code_group[i].transform("mean"), inplace=True)
  data[i].fillna(region_group[i].transform("mean"), inplace=True)
  data[i].fillna(data[i].mean(), inplace=True)

null_cols_str = ["funder", "scheme_management", "installer"]
for k in null_cols_str:
  data[k].fillna("other", inplace=True)

data["construction_year"].fillna(data.groupby(['region', 'district_code'])["construction_year"].transform("median"), inplace=True)
data["construction_year"].fillna(data.groupby(['region'])["construction_year"].transform("median"), inplace=True)
data["construction_year"].fillna(data.groupby(['district_code'])["construction_year"].transform("median"), inplace=True)
data["construction_year"].fillna(data["construction_year"].median(), inplace=True)

data.isnull().sum()

id                           0
amount_tsh                   0
date_recorded                0
funder                       0
gps_height                   0
installer                    0
longitude                    0
latitude                     0
wpt_name                     0
num_private                  0
basin                        0
subvillage                 371
region                       0
region_code                  0
district_code                0
lga                          0
ward                         0
population                   0
public_meeting            3334
recorded_by                  0
scheme_management            0
scheme_name              28166
permit                    3056
construction_year            0
extraction_type              0
extraction_type_group        0
extraction_type_class        0
management                   0
management_group             0
payment                      0
payment_type                 0
water_quality                0
quality_

In [37]:
#reduce number sizes
large_num_feat=['amount_tsh', 'gps_height', 'population']
scaler = MinMaxScaler(feature_range=(0,20))
data[large_num_feat] = scaler.fit_transform(data[large_num_feat])

In [38]:
print(data.isnull().sum())

id                           0
amount_tsh                   0
date_recorded                0
funder                       0
gps_height                   0
installer                    0
longitude                    0
latitude                     0
wpt_name                     0
num_private                  0
basin                        0
subvillage                 371
region                       0
region_code                  0
district_code                0
lga                          0
ward                         0
population                   0
public_meeting            3334
recorded_by                  0
scheme_management            0
scheme_name              28166
permit                    3056
construction_year            0
extraction_type              0
extraction_type_group        0
extraction_type_class        0
management                   0
management_group             0
payment                      0
payment_type                 0
water_quality                0
quality_

In [39]:
data['date_recorded'] = pd.to_datetime(data['date_recorded'])
data['no_working_years'] = data.date_recorded.dt.year - data.construction_year
data.no_working_years.head(5)

0    12.0
1     3.0
2     4.0
3    27.0
4     8.0
Name: no_working_years, dtype: float64

In [40]:
redundant_features=['installer', 'date_recorded','construction_year','wpt_name','num_private','subvillage','region_code','recorded_by','management_group',"extraction_type_group",'extraction_type_class','scheme_name','payment','quality_group','quantity_group','source_type', 'source_class', 'waterpoint_type_group','ward','public_meeting','permit']
#remove redundant features
#remove date_recorded and construction_year as working years has been added
#remove extraction_type_group and extraction_type_class as extraction_type has more classes
#remove scheme_name as each class has small number of data
#remove payment as payment and payment_type has almost same classes
#remove quality_group as quality_group and water_quality has almost same classes and water_quality has more classes
#remove quantity_group as quantity_group and quantity has same classes
#remove source_type and source_class as source is almost same as those
#remove waterpoint_type_group as waterpoint_type_group and waterpoint_type have almost same classes
#remove public_meeting as public_meeting has almost same distribution with status_group
#remove permit as permit has almost same distribution with status_group
data.drop(redundant_features,axis=1,inplace=True)

data.isnull().sum()

id                   0
amount_tsh           0
funder               0
gps_height           0
longitude            0
latitude             0
basin                0
region               0
district_code        0
lga                  0
population           0
scheme_management    0
extraction_type      0
management           0
payment_type         0
water_quality        0
quantity             0
source               0
waterpoint_type      0
no_working_years     0
dtype: int64

In [41]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 59400 non-null  int64  
 1   amount_tsh         59400 non-null  float64
 2   funder             59400 non-null  object 
 3   gps_height         59400 non-null  float64
 4   longitude          59400 non-null  float64
 5   latitude           59400 non-null  float64
 6   basin              59400 non-null  object 
 7   region             59400 non-null  object 
 8   district_code      59400 non-null  int64  
 9   lga                59400 non-null  object 
 10  population         59400 non-null  float64
 11  scheme_management  59400 non-null  object 
 12  extraction_type    59400 non-null  object 
 13  management         59400 non-null  object 
 14  payment_type       59400 non-null  object 
 15  water_quality      59400 non-null  object 
 16  quantity           594

In [42]:
data.head()

,id,amount_tsh,funder,gps_height,longitude,latitude,basin,region,district_code,lga,population,scheme_management,extraction_type,management,payment_type,water_quality,quantity,source,waterpoint_type,no_working_years
0,69572,0.342846,Roman,10.349650,34.938093,-9.856322,Lake Nyasa,Iringa,5,Ludewa,0.070822,VWC,gravity,vwc,annually,soft,enough,spring,communal standpipe,12.0
1,8776,0.021397,Grumeti,10.412587,34.698766,-2.147466,Lake Victoria,Mara,2,Serengeti,0.182957,Other,gravity,wug,never pay,soft,insufficient,rainwater harvesting,communal standpipe,3.0
2,34310,0.001417,Lottery Club,5.426573,37.460664,-3.821329,Pangani,Manyara,4,Simanjiro,0.163284,VWC,gravity,vwc,per bucket,soft,enough,dam,communal standpipe multiple,4.0
3,67743,0.005729,Unicef,2.468531,38.486161,-11.155298,Ruvuma / Southern Coast,Mtwara,63,Nanyumbu,0.037378,VWC,submersible,vwc,never pay,soft,dry,machine dbh,communal standpipe multiple,27.0
4,19728,0.079573,Action In A,8.024794,31.130847,-1.825359,Lake Victoria,Kagera,1,Karagwe,0.207517,other,gravity,other,never pay,soft,seasonal,rainwater harvesting,communal standpipe,8.0


In [43]:
#making all category class names lowercase
data.waterpoint_type = data.waterpoint_type.str.lower()
data.funder = data.funder.str.lower()
#data.installer = data.installer.str.lower()
data.basin = data.basin.str.lower()
data.region = data.region.str.lower()
data.source = data.source.str.lower()
data.lga = data.lga.str.lower()
data.management = data.management.str.lower()
data.quantity = data.quantity.str.lower()
data.water_quality = data.water_quality.str.lower()
data.payment_type=data.payment_type.str.lower()
data.extraction_type=data.extraction_type.str.lower()
data.scheme_management=data.scheme_management.str.lower()

In [44]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 59400 non-null  int64  
 1   amount_tsh         59400 non-null  float64
 2   funder             59400 non-null  object 
 3   gps_height         59400 non-null  float64
 4   longitude          59400 non-null  float64
 5   latitude           59400 non-null  float64
 6   basin              59400 non-null  object 
 7   region             59400 non-null  object 
 8   district_code      59400 non-null  int64  
 9   lga                59400 non-null  object 
 10  population         59400 non-null  float64
 11  scheme_management  59400 non-null  object 
 12  extraction_type    59400 non-null  object 
 13  management         59400 non-null  object 
 14  payment_type       59400 non-null  object 
 15  water_quality      59400 non-null  object 
 16  quantity           594

In [45]:
pd.DataFrame(data).to_csv("preprocced_data.csv",index=False)
#pd.DataFrame(data).to_csv("preprocced_test_data.csv",index=False)